**Instalações necessárias**
- FASE 1: Instalação do python no notebook do colab.

In [ ]:
!pip install -U -q PyDrive

**Importações necessárias**
- FASE 2: Importações das bibliotecas a serem utilizadas.

In [ ]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import warnings
import shutil
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import plotly.express as px

**Autenticação de segurança**
- FASE 3: Verificação de acesso do email INTELI para acessar a base de dados fornecida pelo Banco Pan. Importante para a segurança dos dados.

In [ ]:
# Autenticação para acesso aos dados
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Download dos Dados**
- FASE 4: Realização do download dos dados da base para esse notebook do Colab, para a visualização e manipulação.

In [ ]:
# Download dos dados para esta máquina virtual
downloaded = drive.CreateFile({'id': '1tVSvbYcKDgwdnm3OxbWaSIKXHZYEapyU'})
downloaded.GetContentFile("book_variaveis_pan_inteli_v4.zip")
shutil.unpack_archive("book_variaveis_pan_inteli_v4.zip", "./")

In [ ]:
df = pd.read_csv("book_variaveis_pan_inteli_v4.csv")

**Inicio do tratamento dos Dados**
- FASE 5: Seleção de Safra e tratamento inicial das colunas com dados nulos/vazios, como o modelo de treinamento não aceita dados nulos ou strings, essa fase se torna de suma importância e deve ser realizada com calma.
- Isso está sendo feito para a melhor filtragem de dados, e para não consumirmos uma enorme quantidade de processamento.


In [ ]:
#Seleciona uma safra, tira possiveis duplicados e troca o NaN por 0.

df_202104 = df.loc[df['anomes'] == 202104].drop_duplicates(keep='first').fillna(0)
df_202104


,anomes,num_cpf_hash,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,vlr_renda,cod_rating,ind_atrito,ind_engaj,ind_novo_cli
0,202104,fffff8b0db8eff291be8b83f8885f52c52782bb42c3c4b...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.0,0.0,0.0
1,202104,ffffd54b45ec46113523184fc07185a0d5cbfa876a07ba...,35943.74,5815.50,0.0,377.0,1.0,0.0,24.0,0.0,10.0,0,A,0.0,1.0,0.0
2,202104,ffffd47a92b3e4291c013033ae528708a19eaede50f78e...,6288.22,0.00,0.0,257.0,0.0,0.0,22.0,0.0,4.0,0,0,0.0,0.0,0.0
3,202104,ffffc102ddd37ec29e985a4564e85a2bace79a85ebff5e...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,0,0.0,0.0,0.0
4,202104,ffffbd4a3d42a12e07b1202d68c33d43220c42c8a55160...,1238.93,1400.00,0.0,773.0,2.0,0.0,21.0,0.0,0.0,0,A,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999992,202104,00003a6cf049e4fe3416c1a55d1fc1aac613956ecd5d7a...,0.00,0.00,0.0,382.0,1.0,0.0,1.0,0.0,0.0,0,0,0.0,0.0,0.0
999993,202104,000038baed6e8076adec7272523211faaec7bf8b30b1e1...,30717.14,1848.06,0.0,578.0,1.0,0.0,19.0,0.0,0.0,0,A,0.0,1.0,0.0
999994,202104,00003671431c71ae1a84ec138b63c7a6e465f9616af5ea...,2352.13,1443.31,0.0,372.0,1.0,0.0,9.0,0.0,1.0,0,E,0.0,1.0,0.0
999995,202104,000031eae3bd9f91ad57acc059876d383658f1ba748195...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.0,0.0,0.0


**Tratando Dados**
- FASE 6: Tratamento de valores em formato String da coluna 'cod_rating'. Como citado no passo anterior, um modelo de treinamento não deve possuir strings, sendo assim é necessario fazer o tratamento na coluna 'cod_rating' transformando a escala que antes era "A, AA, B, C, D, E, F, G,H, HH" para "10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0". **Nota: Já que tratamos os valores nulos de toda os campos anteriormente, não é necessario tratar novamente nesta etapa, desta maneira o valor NaN é considerado o valor 0.**

In [ ]:
# Troca de valores escala em string da coluna cod_rating para a escala em formato númerico
df_202104['cod_rating'] = (df_202104['cod_rating'].replace('HH', 1)
                                                  .replace('H', 2)
                                                  .replace('G', 3)
                                                  .replace('F', 4)
                                                  .replace('E', 5)
                                                  .replace('D', 6)
                                                  .replace('C', 7)
                                                  .replace('B', 8)
                                                  .replace('AA', 9)
                                                  .replace('A', 10))

**Ordenar Dados**
- FASE 7: Para a criação do modelo de treinamento, ordenamos os dados tratados com base na coluna de 'ind_atrito' para poder pegar o maior número de linhas que possua valor na coluna diferente de 0. 
- O ordenamento é feito para facilitar a separação entre os clientes com índice atritado 1 e 0, auxiliando o corte dos dados que serão usados no treinamento do modelo.


In [ ]:
#Ordenação dos dados pela coluna ind_atrito
df_202104 = df_202104.sort_values(by=['ind_atrito'],ascending=False)

**Seleção dos Dados para Preparação**
- FASE 8: Devido a safra selecionada, 260 linhas da base de dados possui valor diferente de 0 na coluna 'ind_atrito', por meio deste dado, estipulamos para o modelo preditivo inicial a seleção de ao todo 520 linhas das base de dados sendo fatiado ao meio (260 linhas) em relação ao valor do indice(1=true; 0=false) e no final realizando a randomização dos dados para que esses venham sem esse tratamento de ordenação para a fase de treinamento. 

In [ ]:
from sklearn.utils import shuffle

#sendo 260 atritados, selecionamos 520 linhas, ou seja, meio a meio
dados_preparados = df_202104[0:520]

#randomiza ordem para evitar vício
dados_preparados  = shuffle(dados_preparados)

**Relatório dos Dados em Gráfico**
- FASE 9: Criação do gráfico de valor do score por indice de atrito, para checar se há relação das pessoas que possuem maior score terem um indice maior de atrito.

In [ ]:
#Criação do gráfico com seus repectivos dados
fig = px.area(dados_preparados, x='ind_atrito', y='vlr_score', height=300, width=600)
#Atualização dos gráficos
fig.update_xaxes(type='category')
#Vizualiação do gráfico
fig.show()

**Seleção das Colunas dos Dados**
- FASE 10: Seleção de todas as colunas escolhidas no foco de treinamento da IA. Esses campos foram escolhidos por meio da estimativa de existir correlação entre algumas colunas 'qtd_reclm', 'num_atend_atrs', 'num_atend', 'ind_atrito', 'cod_rating' e 'vlr_score' que ajudaram na fase de treinamento, afim de gerar o modelo preditivo.
- Não estão sendo usadas todas as colunas pois algumas delas poderiam gerar vieses nos resultados e assim, o treinamento não seria tão efetivo.

In [ ]:
#Colunas escolhidas da base de dados para serem utilizadas
colunas_para_treinamento = ['qtd_reclm','num_atend_atrs','num_atend','ind_atrito','cod_rating', 'vlr_score']

